In [1]:
import torch
import torch.nn as nn
import cv2
import csv
import json
import torchvision.models as models
import numpy as np
import pandas as pd
import statistics
# from object_detection import ObjectDetection
import math
import subprocess
from tqdm import tqdm
from torchvision import transforms
# import maths

In [2]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5m', pretrained=True)
model.eval();

class Image_Classifier(nn.Module):
    def init(self):
        super().init()
        self.model = nn.Sequential(
             Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1)),
             ReLU(),
             Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1)),
             ReLU(),
             MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
             Flatten(start_dim=1, end_dim=-1),
             Dropout(p=0.25, inplace=False),
             Linear(in_features=6272, out_features=132, bias=True),
             ReLU(),
             Dropout(p=0.5, inplace=False),
             Linear(in_features=132, out_features=11, bias=True),
        )

    def forward(self, x):
        return self.model(x)

color_classifier = torch.load("color.pt")
color_classifier = color_classifier.cuda()
color_classifier.eval()

body_classifier = torch.load('model.pt')

Using cache found in C:\Users\ahmed/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-4-18 Python-3.9.13 torch-2.0.0+cu118 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 6144MiB)



requirements: C:\Users\ahmed\.cache\torch\hub\requirements.txt not found, check failed.


Fusing layers... 
YOLOv5m summary: 290 layers, 21172173 parameters, 0 gradients
Adding AutoShape... 


In [12]:
"""
    SORT: A Simple, Online and Realtime Tracker
    Copyright (C) 2016-2020 Alex Bewley alex@bewley.ai

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <http://www.gnu.org/licenses/>.
"""
from __future__ import print_function

import os
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from skimage import io

import glob
import time
import argparse
from filterpy.kalman import KalmanFilter

np.random.seed(0)


def linear_assignment(cost_matrix):
  try:
    import lap
    _, x, y = lap.lapjv(cost_matrix, extend_cost=True)
    return np.array([[y[i],i] for i in x if i >= 0]) #
  except ImportError:
    from scipy.optimize import linear_sum_assignment
    x, y = linear_sum_assignment(cost_matrix)
    return np.array(list(zip(x, y)))


def iou_batch(bb_test, bb_gt):
  """
  From SORT: Computes IOU between two bboxes in the form [x1,y1,x2,y2]
  """
  bb_gt = np.expand_dims(bb_gt, 0)
  bb_test = np.expand_dims(bb_test, 1)
  
  xx1 = np.maximum(bb_test[..., 0], bb_gt[..., 0])
  yy1 = np.maximum(bb_test[..., 1], bb_gt[..., 1])
  xx2 = np.minimum(bb_test[..., 2], bb_gt[..., 2])
  yy2 = np.minimum(bb_test[..., 3], bb_gt[..., 3])
  w = np.maximum(0., xx2 - xx1)
  h = np.maximum(0., yy2 - yy1)
  wh = w * h
  o = wh / ((bb_test[..., 2] - bb_test[..., 0]) * (bb_test[..., 3] - bb_test[..., 1])                                      
    + (bb_gt[..., 2] - bb_gt[..., 0]) * (bb_gt[..., 3] - bb_gt[..., 1]) - wh)                                              
  return(o)  


def convert_bbox_to_z(bbox):
  """
  Takes a bounding box in the form [x1,y1,x2,y2] and returns z in the form
    [x,y,s,r] where x,y is the centre of the box and s is the scale/area and r is
    the aspect ratio
  """
  w = bbox[2] - bbox[0]
  h = bbox[3] - bbox[1]
  x = bbox[0] + w/2.
  y = bbox[1] + h/2.
  s = w * h    #scale is just area
  r = w / float(h)
  return np.array([x, y, s, r]).reshape((4, 1))


def convert_x_to_bbox(x,score=None):
  """
  Takes a bounding box in the centre form [x,y,s,r] and returns it in the form
    [x1,y1,x2,y2] where x1,y1 is the top left and x2,y2 is the bottom right
  """
  w = np.sqrt(x[2] * x[3])
  h = x[2] / w
  if(score==None):
    return np.array([x[0]-w/2.,x[1]-h/2.,x[0]+w/2.,x[1]+h/2.]).reshape((1,4))
  else:
    return np.array([x[0]-w/2.,x[1]-h/2.,x[0]+w/2.,x[1]+h/2.,score]).reshape((1,5))


class KalmanBoxTracker(object):
  """
  This class represents the internal state of individual tracked objects observed as bbox.
  """
  count = 0
  def __init__(self,bbox):
    """
    Initialises a tracker using initial bounding box.
    """
    #define constant velocity model
    self.kf = KalmanFilter(dim_x=7, dim_z=4) 
    self.kf.F = np.array([[1,0,0,0,1,0,0],[0,1,0,0,0,1,0],[0,0,1,0,0,0,1],[0,0,0,1,0,0,0],  [0,0,0,0,1,0,0],[0,0,0,0,0,1,0],[0,0,0,0,0,0,1]])
    self.kf.H = np.array([[1,0,0,0,0,0,0],[0,1,0,0,0,0,0],[0,0,1,0,0,0,0],[0,0,0,1,0,0,0]])

    self.kf.R[2:,2:] *= 10.
    self.kf.P[4:,4:] *= 1000. #give high uncertainty to the unobservable initial velocities
    self.kf.P *= 10.
    self.kf.Q[-1,-1] *= 0.01
    self.kf.Q[4:,4:] *= 0.01

    self.kf.x[:4] = convert_bbox_to_z(bbox)
    self.time_since_update = 0
    self.id = KalmanBoxTracker.count
    KalmanBoxTracker.count += 1
    self.history = []
    self.hits = 0
    self.hit_streak = 0
    self.age = 0

  def update(self,bbox):
    """
    Updates the state vector with observed bbox.
    """
    self.time_since_update = 0
    self.history = []
    self.hits += 1
    self.hit_streak += 1
    self.kf.update(convert_bbox_to_z(bbox))

  def predict(self):
    """
    Advances the state vector and returns the predicted bounding box estimate.
    """
    if((self.kf.x[6]+self.kf.x[2])<=0):
      self.kf.x[6] *= 0.0
    self.kf.predict()
    self.age += 1
    if(self.time_since_update>0):
      self.hit_streak = 0
    self.time_since_update += 1
    self.history.append(convert_x_to_bbox(self.kf.x))
    return self.history[-1]

  def get_state(self):
    """
    Returns the current bounding box estimate.
    """
    return convert_x_to_bbox(self.kf.x)


def associate_detections_to_trackers(detections,trackers,iou_threshold = 0.3):
  """
  Assigns detections to tracked object (both represented as bounding boxes)

  Returns 3 lists of matches, unmatched_detections and unmatched_trackers
  """
  if(len(trackers)==0):
    return np.empty((0,2),dtype=int), np.arange(len(detections)), np.empty((0,5),dtype=int)

  iou_matrix = iou_batch(detections, trackers)

  if min(iou_matrix.shape) > 0:
    a = (iou_matrix > iou_threshold).astype(np.int32)
    if a.sum(1).max() == 1 and a.sum(0).max() == 1:
        matched_indices = np.stack(np.where(a), axis=1)
    else:
      matched_indices = linear_assignment(-iou_matrix)
  else:
    matched_indices = np.empty(shape=(0,2))

  unmatched_detections = []
  for d, det in enumerate(detections):
    if(d not in matched_indices[:,0]):
      unmatched_detections.append(d)
  unmatched_trackers = []
  for t, trk in enumerate(trackers):
    if(t not in matched_indices[:,1]):
      unmatched_trackers.append(t)

  #filter out matched with low IOU
  matches = []
  for m in matched_indices:
    if(iou_matrix[m[0], m[1]]<iou_threshold):
      unmatched_detections.append(m[0])
      unmatched_trackers.append(m[1])
    else:
      matches.append(m.reshape(1,2))
  if(len(matches)==0):
    matches = np.empty((0,2),dtype=int)
  else:
    matches = np.concatenate(matches,axis=0)

  return matches, np.array(unmatched_detections), np.array(unmatched_trackers)


class Sort(object):
  def __init__(self, max_age=1, min_hits=3, iou_threshold=0.3):
    """
    Sets key parameters for SORT
    """
    self.max_age = max_age
    self.min_hits = min_hits
    self.iou_threshold = iou_threshold
    self.trackers = []
    self.frame_count = 0

  def update(self, dets=np.empty((0, 5))):
    """
    Params:
      dets - a numpy array of detections in the format [[x1,y1,x2,y2,score],[x1,y1,x2,y2,score],...]
    Requires: this method must be called once for each frame even with empty detections (use np.empty((0, 5)) for frames without detections).
    Returns the a similar array, where the last column is the object ID.

    NOTE: The number of objects returned may differ from the number of detections provided.
    """
    self.frame_count += 1
    # get predicted locations from existing trackers.
    trks = np.zeros((len(self.trackers), 5))
    to_del = []
    ret = []
    for t, trk in enumerate(trks):
      pos = self.trackers[t].predict()[0]
      trk[:] = [pos[0], pos[1], pos[2], pos[3], 0]
      if np.any(np.isnan(pos)):
        to_del.append(t)
    trks = np.ma.compress_rows(np.ma.masked_invalid(trks))
    for t in reversed(to_del):
      self.trackers.pop(t)
    matched, unmatched_dets, unmatched_trks = associate_detections_to_trackers(dets,trks, self.iou_threshold)

    # update matched trackers with assigned detections
    for m in matched:
      self.trackers[m[1]].update(dets[m[0], :])

    # create and initialise new trackers for unmatched detections
    for i in unmatched_dets:
        trk = KalmanBoxTracker(dets[i,:])
        self.trackers.append(trk)
    i = len(self.trackers)
    for trk in reversed(self.trackers):
        d = trk.get_state()[0]
        if (trk.time_since_update < 1) and (trk.hit_streak >= self.min_hits or self.frame_count <= self.min_hits):
          ret.append(np.concatenate((d,[trk.id+1])).reshape(1,-1)) # +1 as MOT benchmark requires positive
        i -= 1
        # remove dead tracklet
        if(trk.time_since_update > self.max_age):
          self.trackers.pop(i)
    if(len(ret)>0):
      return np.concatenate(ret)
    return np.empty((0,5))

def parse_args():
    """Parse input arguments."""
    parser = argparse.ArgumentParser(description='SORT demo')
    parser.add_argument('--display', dest='display', help='Display online tracker output (slow) [False]',action='store_true')
    parser.add_argument("--seq_path", help="Path to detections.", type=str, default='data')
    parser.add_argument("--phase", help="Subdirectory in seq_path.", type=str, default='train')
    parser.add_argument("--max_age", 
                        help="Maximum number of frames to keep alive a track without associated detections.", 
                        type=int, default=1)
    parser.add_argument("--min_hits", 
                        help="Minimum number of associated detections before track is initialised.", 
                        type=int, default=3)
    parser.add_argument("--iou_threshold", help="Minimum IOU for match.", type=float, default=0.3)
    args = parser.parse_args()
    return args

# if __name__ == '__main__':
    
#     video_path = 'traffic-27260.mp4'
#     output_path = 'test.mp4'
#     cap = cv2.VideoCapture(video_path)
#      #Get the video frame rate and dimensions
#     fps = cap.get(cv2.CAP_PROP_FPS)
#     frame_width =  int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
#     frame_height =  int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

#     ffmpeg_cmd = f"ffmpeg -y -f rawvideo -pix_fmt bgr24 -s {frame_width}x{frame_height} -r {fps} -i - -c:v libx264 -preset fast -crf 30 -pix_fmt nv12 -an -vcodec libx264 {output_path}"

#     # Open the output file for writing 
#     output_file = subprocess.Popen(ffmpeg_cmd.split(' '), stdin=subprocess.PIPE)
# #     if not os.path.exists('output'):
# #         os.makedirs('output')
#     mot_tracker = Sort(max_age=30,
#                        min_hits=60) 
#     frame_cut = 0
#     while True:
#         ret, frame = cap.read()
#         if not ret:
#             print('asssss')
#             break      

#         frame_model = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

#         results = model(frame_model[frame_cut:])
# #         results = results.xyxy[0][:,:4].cpu()
#         track_bbs_ids = mot_tracker.update(results.xyxy[0][:,:4].cpu())
    
#         for x1,y1,x2,y2, obj_id in track_bbs_ids:
#             print(x1,y1,x2,y2, obj_id)
#             cx1 =  int((x1 + x2 ) / 2)
#             cy1 =  int((y1 + y2 ) / 2)
#             cv2.putText(frame, str(obj_id), (cx1, cy1 - 7), 0, 1, (255, 255, 255), 2)
# #             cv2.rectangle(frame, int(x1), int(y1), int(x2), int(y2), (255, 174, 66))

#         output_file.stdin.write(frame.tobytes())     
#     cap.release()
#     output_file.stdin.close()
#     output_file.wait()

In [13]:
if __name__ == '__main__':
    video_path = 'traffic-27260.mp4'
    output_path = 'test.mp4'
    cap = cv2.VideoCapture(video_path)

    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    ffmpeg_cmd = f"ffmpeg -y -f rawvideo -pix_fmt bgr24 -s {frame_width}x{frame_height} -r {fps} -i - -c:v libx264 -preset fast -crf 30 -pix_fmt nv12 -an -vcodec libx264 {output_path}"

    output_file = subprocess.Popen(ffmpeg_cmd.split(' '), stdin=subprocess.PIPE)

    mot_tracker = Sort(max_age=30, min_hits=60) 

    object_dict = {}

    frame_cut = 0
    frame_count = 0
    clf_state = False
    while True:
        ret, frame = cap.read()
        if not ret:
            print('Video processing completed')
            break

        frame_model = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        results = model(frame_model[frame_cut:])

        track_bbs_ids = mot_tracker.update(results.xyxy[0][:,:4].cpu())
        for x1, y1, x2, y2, obj_id in track_bbs_ids:
            cx1 = int((x1 + x2) / 2)
            cy1 = int((y1 + y2) / 2)
            width = abs(x2 - x1)
            height = abs(y2 - y1)
            diagonal = math.sqrt(width**2 + height**2)
            print(diagonal,obj_id,frame_count)
            if diagonal > 60 :
                try:
                    car_image = frame_model[int(y1):int(y2), int(x1):int(x2)]
                    transform = transforms.Compose([transforms.ToTensor(),
                                                    transforms.Resize((32,32)),])
                    car_image = transform(car_image)
                    car_image = car_image.cuda()
                except:
                    print('no object detected')
                    continue
                with torch.no_grad():
                    color_output = color_classifier(car_image.unsqueeze(0))
                    color_prediction = torch.argmax(color_output).item()
                    color_name = ['black','blue','brown','green','grey','orange','pink','purple','red','white','yellow']
                    color_class_name = color_name[color_prediction]

                try:
                    body_car_image = frame_model[int(y1):int(y2), int(x1):int(x2)]
                    transform = transforms.Compose([transforms.ToTensor(),
                                                    transforms.Resize((256,256)),])
                    body_car_image = transform(body_car_image)
                    body_car_image = body_car_image.cuda()
                except:
                    print('no object detected')
                    continue

                with torch.no_grad():
                    body_output = body_classifier(body_car_image.unsqueeze(0))
                    body_prediction = torch.argmax(body_output).item()
                    body_name = ['Heavy-Duty', 'Lorry', 'Luxury', 'Pickup', 'SUV', 'Sedan', 'Van']
                    body_class_name = body_name[body_prediction]
                    clf_state = True

            
                if obj_id not in object_dict:
                    object_dict[obj_id] = {
                        'bboxes': [(frame_count, x1, y1, x2, y2)],
                        'last_seen_frame': 0,
                        'color_classifier_preds': [color_prediction],
                        'body_classifier_preds': [body_prediction]
                    }
                else:
                    object_dict[obj_id]['bboxes'].append((frame_count, x1, y1, x2, y2))
                    object_dict[obj_id]['color_classifier_preds'].append(color_prediction)
                    object_dict[obj_id]['body_classifier_preds'].append(body_prediction)

                object_dict[obj_id]['last_seen_frame'] = frame_count

                # Calculate the mode prediction of the classifier for the tracked object
                color_mode_pred = statistics.mode(object_dict[obj_id]['color_classifier_preds'])
                object_dict[obj_id]['color_mode_pred'] = color_mode_pred

                body_mode_pred = statistics.mode(object_dict[obj_id]['body_classifier_preds'])
                object_dict[obj_id]['body_mode_pred'] = body_mode_pred

            cv2.putText(frame, str(obj_id), (cx1, cy1), 0, 1, (255, 255, 255), 2)
            if clf_state == True:
                cv2.putText(frame, color_name[color_mode_pred], (int(x1), int(y1)), cv2.FONT_HERSHEY_SIMPLEX, 1, (203, 192, 255), 2)
                cv2.putText(frame,  body_name[body_mode_pred], (int(cx1), int(y2)), cv2.FONT_HERSHEY_SIMPLEX, 1, (203, 192, 255), 2)

        output_file.stdin.write(frame.tobytes())

        for obj_id, obj_data in list(object_dict.items()):
            if frame_count - obj_data['last_seen_frame'] > mot_tracker.max_age:
                object_dict.pop(obj_id)

        frame_count += 1
    
    cap.release()
    output_file.stdin.close()
    output_file.wait()
    
    with open("object_tracks.json", "w") as f:
        json.dump(object_dict, f, indent=4)
        

42.80021185201267 9.0 0
21.49662471670597 8.0 0
63.085935955303974 7.0 0
31.313653408955677 6.0 0
34.26100678369351 5.0 0
43.950760614700584 4.0 0
63.86661918337585 3.0 0
63.61982563598417 2.0 0
65.47093779355804 1.0 0
33.86037526893839 10.0 1
42.85108792003954 9.0 1
74.34135061439748 7.0 1
32.389850094862766 6.0 1
36.60685174746027 5.0 1
43.88887945356696 4.0 1
64.42240115817192 3.0 1
66.96923919325158 2.0 1
69.90006166081649 1.0 1
32.47025523836943 10.0 2
43.65352700000607 9.0 2
21.16991995939291 8.0 2
75.09704358473557 7.0 2
35.53028593977205 6.0 2
37.427009947761256 5.0 2
45.218904177055116 4.0 2
63.992729375354095 3.0 2
67.76633571810424 2.0 2
69.76562729779168 1.0 2
32.14739346135916 10.0 3
43.873133049762764 9.0 3
21.119098815284527 8.0 3
75.18231882365822 7.0 3
36.41002119626078 6.0 3
37.62587532154374 5.0 3
45.633979812103334 4.0 3
63.96672951261923 3.0 3
67.93904754283876 2.0 3
69.69580680607072 1.0 3
32.98908479436796 10.0 4
44.82005747837396 9.0 4
67.76703894265577 7.0 4
36

29.541767402982206 12.0 41
30.177214506500874 8.0 41
64.446397909509 7.0 41
48.700101115452405 6.0 41
48.04197253427111 5.0 41
73.53707306539403 4.0 41
49.5893079183329 3.0 41
129.27518532008088 2.0 41
144.43234119359562 1.0 41
29.27050330264817 12.0 42
30.36439671211346 8.0 42
64.36156428190864 7.0 42
49.05750681061039 6.0 42
48.79075567501092 5.0 42
75.57474995711071 4.0 42
48.63277640560808 3.0 42
132.6751980000027 2.0 42
150.31869537735864 1.0 42
28.94573679518949 12.0 43
64.63573441050916 7.0 43
48.95085337696466 6.0 43
49.17119270403202 5.0 43
76.57925658815928 4.0 43
49.34686587776042 3.0 43
139.08722531432437 2.0 43
155.03833992083813 1.0 43
28.73686277281887 12.0 44
64.00859011371551 7.0 44
48.92907330225712 6.0 44
50.685643893723565 5.0 44
78.61253539186923 4.0 44
48.795918194852995 3.0 44
145.05524668913412 2.0 44
159.3947892792901 1.0 44
32.92595035009202 13.0 45
29.02974831867051 12.0 45
63.42871117203238 7.0 45
49.056124433394395 6.0 45
50.844070715202484 5.0 45
78.402076

42.28075194456756 12.0 103
114.63339347737384 6.0 103
102.92879355389525 5.0 103
438.3864954585517 4.0 103
42.408823426780096 12.0 104
117.42203282765931 6.0 104
105.62477273611773 5.0 104
440.9991163471473 4.0 104
42.68524385561648 12.0 105
120.34201247887904 6.0 105
106.92722021532127 5.0 105
419.5895485369856 4.0 105
42.89972667821921 12.0 106
122.43205187316431 6.0 106
107.8562973128789 5.0 106
405.42405957956146 4.0 106
43.234030780268455 12.0 107
44.88598028167193 8.0 107
124.83795188615541 6.0 107
110.84200991594349 5.0 107
388.31912288270564 4.0 107
43.668086579595105 12.0 108
45.31918511990731 8.0 108
126.90844291606435 6.0 108
114.10932466461394 5.0 108
43.82041963471574 12.0 109
44.51511523945834 8.0 109
130.80229243857912 6.0 109
116.39103506730717 5.0 109
44.46955667134623 12.0 110
44.23608975427137 8.0 110
136.37966606768686 6.0 110
120.57042234188988 5.0 110
44.57644911488926 12.0 111
44.36981192093227 8.0 111
140.23969306080303 6.0 111
125.40956617298826 5.0 111
44.5895

37.00405384305095 23.0 190
73.59956745261776 12.0 190
65.46508854749919 8.0 190
36.84103088870166 23.0 191
75.45874148725349 12.0 191
66.44909557422022 8.0 191
62.21361298828882 26.0 192
36.931304447543056 23.0 192
76.50553176351863 12.0 192
68.48972597556345 8.0 192
63.51861102533426 26.0 193
37.01450763432185 23.0 193
77.25319312070492 12.0 193
69.65008046187594 8.0 193
64.33779544646946 26.0 194
36.78696014243645 23.0 194
77.66399784026176 12.0 194
70.90491292318131 8.0 194
64.96292959487454 26.0 195
36.64414429490698 23.0 195
78.2169371801835 12.0 195
71.59181534990945 8.0 195
65.51084212958672 26.0 196
37.392591837883984 23.0 196
79.19192640122094 12.0 196
72.07953876967947 8.0 196
65.90783367848536 26.0 197
37.91754932739154 23.0 197
79.90956836425033 12.0 197
72.44450171662669 8.0 197
67.15128662409793 26.0 198
38.95278259667851 23.0 198
81.56156952070573 12.0 198
72.79054462518505 8.0 198
69.06645552213058 26.0 199
39.63720004875799 23.0 199
83.15290236158091 12.0 199
73.601674

109.45961442790019 37.0 251
77.05800048276687 36.0 251
197.6080478283774 33.0 251
427.71188624637455 26.0 251
63.971594632590694 23.0 251
275.5388594134888 8.0 251
111.23822605058827 37.0 252
78.08554289044339 36.0 252
196.18189939962946 33.0 252
407.8082456764876 26.0 252
65.58337511317954 23.0 252
287.19439591443177 8.0 252
111.8094400260483 37.0 253
80.19821592848834 36.0 253
187.77803389520074 33.0 253
66.49443540123445 23.0 253
303.7444757465569 8.0 253
114.4500281373099 37.0 254
80.76135540661912 36.0 254
173.70782778055968 33.0 254
66.78025914159356 23.0 254
322.72305928159886 8.0 254
116.69307006543598 37.0 255
81.94041143998479 36.0 255
157.32660410903685 33.0 255
67.38353137763525 23.0 255
342.7601022147397 8.0 255
119.63901177184002 37.0 256
84.19358578567686 36.0 256
67.99398045822241 23.0 256
365.7239188429682 8.0 256
121.74971673283267 37.0 257
85.82276315910836 36.0 257
68.47628053043024 23.0 257
382.1256800078956 8.0 257
125.39315694726301 37.0 258
86.06743618947925 36.

50.33605049323622 46.0 322
163.17652482452903 43.0 322
57.20387074667519 41.0 322
60.66256227009889 40.0 322
458.0046761980422 23.0 322
50.30029818393983 46.0 323
166.57049303031712 43.0 323
58.6402242637046 41.0 323
60.72316330183369 40.0 323
445.6998431985925 23.0 323
no object detected
50.24409984854951 46.0 324
169.00452058264867 43.0 324
59.642070281434854 41.0 324
60.77960888034179 40.0 324
436.5588355327176 23.0 324
no object detected
50.11148479242872 46.0 325
170.73094732409817 43.0 325
60.31380109280669 41.0 325
60.851870826715434 40.0 325
429.4607495115484 23.0 325
no object detected
49.99062969762633 46.0 326
171.942436261399 43.0 326
60.514064884832734 41.0 326
60.90546870474505 40.0 326
423.9786889999804 23.0 326
no object detected
49.902909424824614 46.0 327
172.83094646472196 43.0 327
60.6562648158084 41.0 327
60.94801013077303 40.0 327
419.8218638380116 23.0 327
no object detected
49.83280925277973 46.0 328
173.45485035170924 43.0 328
60.73583271908318 41.0 328
60.9689

In [ ]:
results.xyxy[0][:,:4]

In [ ]:
for x1,y1,x2,y2, obj_id in track_bbs_ids:
    print(x1,y1,x2,y2, obj_id)

In [ ]:
import torchvision.transforms as T
from PIL import Image
transform = T.ToPILImage()
img = transform(car_image)
img.show()
output=classifier(car_image.unsqueeze(0))
prediction = torch.argmax(output).item()
print(prediction)